# Using Teleconnection Indices as Predictors for Atmospheric Parameters #

### Environmental Setup and Data Aggregation/Cleaning ###

In [ ]:
# INITIALIZING LIBRARIES

# The following libraries are used as part of this project

# data modules
import pandas as pd
from cfgrib.xarray_store import open_dataset
import xarray as xr
import numpy as np

# data aggregation libraries
import ecmwf.data as ecdata
from ecmwf.opendata import Client
from pathlib import Path

# plotting modules
from magpye import GeoMap
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import figure
import matplotlib.pyplot as plot
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# statistical packages
from scipy.signal import savgol_filter
import statsmodels.api as sm
import xarray.ufuncs as xu
import scipy.stats as stats
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# climatology packages using meteostat
from meteostat import Point
from meteostat import Stations, Daily, Hourly, Monthly

# misc packages
from prettytable import PrettyTable
import datetime
from datetime import datetime
from os.path import dirname, join

### User input is provided to initialize model ###

In [ ]:
########################################## User Input To Initialze Model #######################################

# Weather parameter input
user_input_wx_var = 'surface_temp'

# Align variable names to be used by the meteostats network
if user_input_wx_var == 'surface_temp':
    climo_indice = 'tavg'
    units = 'deg. F'

if user_input_wx_var == 'surface_wind':
    climo_indice = 'wspd'
    units = 'mph'

if user_input_wx_var == 'surface_pressure':
    climo_indice = 'pres'
    units = 'hpa'

if user_input_wx_var == 'precipitation':
    climo_indice = 'prcp' 
    units = 'mm'

# Location input
user_input_loc = 'Buffalo'

# Season selection
season_sel = 'winter'

### Dictionary of city locations using lat/lon identifiers ###

In [ ]:
############################# Initialize and Clean for Analysis #########################################

# Dictionary of cities - this includes lat/lon, city name and title for each location

cities = {
    'Boston': {
        'latitude': 42.090925,
        'longitude': -71.26435,
        'city_name' : 'Boston',
        'title': 'Boston, MA',
        
    },
    'Chicago': {
        'latitude': 41.862306,
        'longitude': -87.616672,
        'city_name' : 'Chicago',
        'title': 'Chicago, IL',
    },
    'New York': {
        'latitude': 40.812194,
        'longitude': -74.076983,
        'city_name' : 'New York',
        'title': 'New York, NY',
    },
    'Buffalo': {
        'latitude': 42.773739,
        'longitude': -78.786978,
        'city_name' : 'Buffalo',
        'title': 'Buffalo, NY',
    },
    'Pittsburg': {
        'latitude': 40.446786,
        'longitude': -80.015761,
        'city_name' : 'Pittsburg',
        'title': 'Pittsburg, PA',
    },
    'Philadelphia': {
        'latitude': 39.900775,
        'longitude': -75.167453,
        'city_name' : 'Philadelphia',
        'title': 'Philadelphia, PA',
    },
    'Cleveland': {
        'latitude': 41.506022,
        'longitude': -81.699564,
        'city_name' : 'Cleveland',
        'title': 'Cleveland, OH',
    },
    'Cincinnati': {
        'latitude': 39.095442,
        'longitude': -84.516039,
        'city_name' : 'Cincinnati',
        'title': 'Cincinnati, OH',
    },
    'Washington DC': {
        'latitude': 38.907697,
        'longitude': -76.864517,
        'city_name' : 'Washington DC',
        'title': 'Washington DC',
    },
    'Baltimore': {
        'latitude': 39.277969,
        'longitude':-76.622767,
        'city_name' : 'Baltimore',
        'title': 'Baltimore, MD',
    },
    'Nashville': {
        'latitude': 36.166461,
        'longitude':-86.771289,
        'city_name' : 'Nashville',
        'title': 'Nashville, TN',
    },
    'Charlotte': {
        'latitude': 35.225808,
        'longitude': -80.852861,
        'city_name' : 'Charlotte',
        'title': 'Charlotte, NC',
    },
    'Jacksonville': {
        'latitude': 30.323925,
        'longitude': -81.637356,
        'city_name' : 'Jacksonville',
        'title': 'Jacksonville, FL',
    },
    'Tampa Bay': {
        'latitude':27.975967,
        'longitude': -82.50335,
        'city_name' : 'Tampa Bay',
        'title': 'Tampa Bay, FL',
    },
    'Miami': {
        'latitude': 25.957919,
        'longitude': -80.238842,
        'city_name' : 'Miami',
        'title': 'Miami, FL',
    },
    'Green Bay': {
        'latitude': 44.501306,
        'longitude': -88.062167,
        'city_name' : 'Green Bay',
        'title': 'Green Bay, WI',
    },
    'Kansas City': {
        'latitude': 39.048914,
        'longitude': -94.484039,
        'city_name' : 'Kansas City',
        'title': 'Kansas City, MO',
    },
    'Denver': {
        'latitude': 39.743936,
        'longitude': -105.020097,
        'city_name' : 'Denver',
        'title': 'Denver, CO',
    },
    'Seattle': {
        'latitude': 47.595153,
        'longitude': -122.331625,
        'city_name' : 'Seattle',
        'title': 'Seattle, WA',
    },
    'San Francisco': {
        'latitude': 37.713486,
        'longitude':-122.386256,
        'city_name' : 'San Francisco',
        'title': 'San Francisco, CA',
    },
    
}

##  Data Aggregation and Cleaning ##
#### ESRL PSL monthly teleconnection data is extracted via a url for the ENSO, PDO, AO, NAO, and PNA. This data will be used as part of a multivariate linear analysis to determine predictability of atmospheric variables using these indices over specified locations. The data is first downloaded using the provided url and then saved as a Pandas data frame. The data is then vertically orientated and stacked so the data of the monthly indice becomes the index. This is completed for each of the 5 teleconnection indices. Data is then merged using an outer join technique using the “Date” index to join the different indices together. Missing values are then converted to 'NaN' values. All row values for each column are dropped if a NaN is detected in that row using the 'dropna' function. ####

In [ ]:
# load teleconnection datasets from ESRL PSL for ENSO, PDO, NAO, AO, and PNA

enso = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/censo.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=2, engine='python')
pdo = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/pdo.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=14, engine='python')
nao = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/nao.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=3, engine='python')
ao = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/ao.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=3, engine='python')
pna = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/pna.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=3, engine='python')


In [ ]:
# Organize and clean data - ENSO

# create new date column
enso_new=pd.DataFrame()

# create date_ranage - fequency by month
enso_new['Date']=pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")

# set index as date column
enso_new = enso_new.set_index('Date')

# stack so values are vertically integrated for each date
len(enso.loc[:,1:].stack().values)
enso_new['ENSO']=enso.loc[:,1:].stack().values

# only select time frame extending from 1950 to 2021 due to data sample compatability
enso_new = enso_new.loc[(enso_new.index.year >= 1950) & (enso_new.index.year <=2021)]

# print dataframe
enso_new

In [ ]:
# Repeat above steps for each teleconnection indice (NAO, PDO, AO, PNA)

# set empy pandas dataframes
nao_new=pd.DataFrame()
pdo_new=pd.DataFrame()
ao_new=pd.DataFrame()
pna_new=pd.DataFrame()

# Identify data range to accomodate full list of indices
nao_new['Date'] = pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")
pdo_new['Date'] = pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")
ao_new['Date'] = pd.date_range(start=pd.datetime(1950,1,1),end=pd.datetime(2022,12,1),freq="MS")
pna_new['Date'] = pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")

# set index as 'Date'
nao_new = nao_new.set_index('Date')
pdo_new = pdo_new.set_index('Date')
ao_new = ao_new.set_index('Date')
pna_new = pna_new.set_index('Date')

# vertically stack indices to correspond with Date index
nao_new['NAO']=nao.loc[:,1:].stack().values
pdo_new['PDO']=pdo.loc[:,1:].stack().values
ao_new['AO']=ao.loc[:,1:].stack().values
pna_new['PNA']=pna.loc[:,1:].stack().values

# only select time frame extending from 1950 to 2021 due to data sample compatability
nao_new = nao_new.loc[(nao_new.index.year >= 1950) & (nao_new.index.year <=2021)]
pdo_new = pdo_new.loc[(pdo_new.index.year >= 1950) & (pdo_new.index.year <=2021)]
ao_new = ao_new.loc[(ao_new.index.year >= 1950) & (ao_new.index.year <=2021)]
pna_new = pna_new.loc[(pna_new.index.year >= 1950) & (pna_new.index.year <=2021)]

In [ ]:
# dataset is merged using each teleconnection indice. An outer join is performed on the date column
newdf_all = pd.merge(enso_new,pdo_new, how= 'outer',on='Date')
newdf_all = pd.merge(newdf_all,nao_new, how= 'outer',on='Date')
newdf_all = pd.merge(newdf_all,ao_new, how= 'outer',on='Date')
newdf_all = pd.merge(newdf_all,pna_new, how= 'outer',on='Date')

# set nans for values that are numerically indicated as missing
newdf_all['PDO'][newdf_all['PDO'] <= -9.9] = np.nan 
newdf_all['ENSO'][newdf_all['ENSO'] <= -9.9] = np.nan 
newdf_all['NAO'][newdf_all['NAO'] <= -9.9] = np.nan
newdf_all['AO'][newdf_all['AO'] <= -9.9] = np.nan
newdf_all['PNA'][newdf_all['PNA'] <= -9.9] = np.nan

### CLIMATOLOGY CALCULATIONS ###

#### a full climotology is calculated using the Meteostat Python Package. Nearest station approximator is used to find observational stations closest to specified lat/lon location. A station is located that is closest, but also goes as far back as possible, preferably back to 1948 ####

In [ ]:
# full climo for station location extending 1950-2021
# This will need to be customized based on availablility of station location data

# Set time period
start = datetime(1950, 1, 1)
end = datetime(2021, 12, 31)

# nearby stations are aggregated 
stations = Stations()
stations = stations.nearby(cities[user_input_loc]['latitude'],cities[user_input_loc]['longitude'])

# 10 of the closest observation stations are used to determine which station best meets date criteria. 
station = stations.fetch(10)
print("The stations for the locations included:", station)
for a in range(0,10,1):   
        if (station['monthly_start'].dt.year.values[a] <= 1950) & (station['monthly_end'].dt.year.values[a] >= 2021):
            print("this station works")
            station_climo = station.loc[(station.index== station.index[a])]
            break

station = station_climo.index.values[0]

# Get monthly data
data_monthly_climo = Monthly(station, start, end)
data_monthly_climo = data_monthly_climo.fetch()

# index is set to "Date" so data can be merged to existing teleconnection DF
data_monthly_climo.index.names = ['Date']

if climo_indice == 'tavg':
    data_monthly_climo.tavg = (data_monthly_climo .tavg * 9/5) + 32

if climo_indice == 'wspd':
    data_monthly_climo.wspd = (data_monthly_climo.wspd/1.609)

# only relavent weather data (as specified by the user), is extracted
var_monthly_climo = data_monthly_climo[climo_indice]

In [ ]:
# Meteostat station data is merged to teleconnection DF 

newdf_climo = pd.merge(newdf_all,data_monthly_climo[climo_indice], how= 'outer',on='Date')

# dataset with all met data and teleconnection indices
newdf_climo

In [ ]:
# procedure to clean dataset of nans - this is completed for all columns

# drop nan values over all columns
newdf_climo_imp = newdf_climo.dropna()

# confirm all nan values have been removed
print(newdf_climo_imp.isna().sum())

# refined dataset by removing NaNs
newdf_climo_imp

In [ ]:
# Seasonal data aggregation - only months selected for the identified season will be considered

if season_sel == 'winter':
    season_data = newdf_climo_imp.loc[(newdf_climo_imp.index.month.values == 12) |
                                        (newdf_climo_imp.index.month.values == 1) |
                                        (newdf_climo_imp.index.month.values == 2)]
if season_sel == 'spring':
    season_data = newdf_climo_imp.loc[(newdf_climo_imp.index.month.values == 3) |
                                        (newdf_climo_imp.index.month.values == 4) |
                                        (newdf_climo_imp.index.month.values == 5)]
if season_sel == 'summer':
    season_data = newdf_climo_imp.loc[(newdf_climo_imp.index.month.values == 6) |
                                        (newdf_climo_imp.index.month.values == 7) |
                                        (newdf_climo_imp.index.month.values == 8)]
if season_sel == 'fall':
    season_data = newdf_climo_imp.loc[(newdf_climo_imp.index.month.values == 9) |
                                        (newdf_climo_imp.index.month.values == 10) |
                                        (newdf_climo_imp.index.month.values == 11)]
newdf_climo_imp = season_data

In [ ]:
######### Pairplot ##########
sns.pairplot(newdf_climo_imp)


######### Heat Map ##########

# correlate values in teleconnection and atmos variable dataset
newdf_climo_corr = newdf_climo_imp.corr() 

# mask out upper triangle of values                                             
mask = np.triu(np.ones_like(newdf_climo_corr, dtype=bool))

# initiate subplot
fig, ax = plt.subplots(figsize=(12, 10))

cmap = 'coolwarm' # color map to identify +/- correlations

# create heatmap of correlation values                                 
sns.heatmap(newdf_climo_corr, mask=mask,annot=True, fmt=".2f", # two digits after decimal point
          cmap=cmap, vmin=-1, vmax=1, # (3) 
           square=True,linewidth=1)  # (4)

### Machine Learning Technique 1: Multivariate Linear Regression Model ###

In [ ]:
# Set up target and features datasets

# X_data includes the teleconnection indices which is our features dataset
X_data= newdf_climo_imp.drop(climo_indice, axis=1) 

# y_data is our target dataset. This is the met variable specified by the user
y_data = newdf_climo_imp[climo_indice]

# testing and training dataset are split using a 30%/70% split

Xtrain, Xtest, ytrain, ytest = train_test_split(X_data, y_data,
                                                test_size=0.7, train_size=0.3) # 70/30 split used for training and testing

# initiate and run linear model

model = LinearRegression() # instantiate model
model.fit(Xtrain, ytrain)  # fit model to data - needs to be applied to training data
y_model = model.predict(Xtest) # use model to predict target variable (y) using set of feature variables (x)

# testing meterics
r2_scores = model.score(Xtrain, ytrain) # R^2 value
print('r2 (coef of determination) score is',r2_scores) 
print('mean_sqrd_error is==',mean_squared_error(ytest,y_model)) # find RMSE between target testing and modeled dataset
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(ytest,y_model))) # print score

## Multiple Linear Regression Model Performance Analysis ##

In [ ]:
# Stats Summary:

# Alternative method using Statsmodel package to retrieve stats summary

#define response variable
y = newdf_climo_imp[climo_indice]

#define predictor variables
x = newdf_climo_imp.drop(climo_indice, axis=1)

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model_v2 = sm.OLS(y, x).fit()

#view model summary
print(model_v2.summary())

In [ ]:
# Scatter Plot of Observed Vs. Predicted Values

# scatter and best fit using target testing dataset and linear model calibrated using the features testing dataset
ytest_plot = ytest.reset_index(drop=True)
fig,ax=plt.subplots(figsize=(20,10))
plt.grid()
plt.scatter(ytest_plot,y_model)
plt.title("Scatter Plot and Best-Fit of Observed vs. Predicted {}".format(climo_indice))
plt.xlabel('Observed {} ({})'.format(climo_indice, units))
plt.ylabel('Predicted {} ({})'.format(climo_indice, units))
lim = plt.axis()

# best fit using polyfit function
m, b = np.polyfit(ytest_plot,y_model, 1)
plt.plot(ytest_plot, m*ytest_plot+b, color='red')

In [ ]:
# Residuals Analysis

# Plot residuals - scatterplot
fig,ax=plt.subplots()
ax.scatter(y_model,ytest-y_model,color='b')
ax.set_xlabel('Predicted {} ({})'.format(climo_indice, units))
ax.set_ylabel('Residuals')
ax.set_title('Scatter Plot of Residuals')
plt.show()

# Plot residuals - histogram
fig, ax = plt.subplots()
ax.hist(ytest-y_model)
ax.set_xlabel('Residual')
ax.set_ylabel('Frequency')
ax.set_title('Histogram of Residuals')
plt.show()

### Machine Learning Technique 2: Random Forest Regression Model ###

In [ ]:
# Random Forest Regression Model Initiation

# set grid with hyperparameters
rf_grid ={'bootstrap': [True, False], # method for sampling data points (with/without replacement)  
            'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  # num. of levels in each descision tree
            'max_features': ['auto', 'sqrt'],  # max number of features for splitting a node
            'min_samples_leaf': [1, 2, 4],  # min num. of points allowed in a leaf node
            'min_samples_split': [2, 5, 10], # min number of data points placed in a node before the node is split 
            'n_estimators': [50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]} # num. of trees in the forest

# identify model
rf_base = RandomForestRegressor()

# perform grid search with cross validation score
rf_random = RandomizedSearchCV(estimator=rf_base, param_distributions=rf_grid, cv=7) # k-fold cross validation = 7

# fit the model using training data
regress_tree_output = rf_random.fit(Xtrain, ytrain)

regress_tree_output # output printed

# identify best model based on hyperparameter input
model_rf = rf_random.best_estimator_ 
print(model_rf)

# cross validation scores using identified model
score_rf = cross_val_score(model_rf, Xtrain, ytrain, cv=7)
print(score_rf)
score_mean_rf = score_rf.mean()
print(score_mean_rf) # mean score of model performance output

model_rf.fit(Xtrain, ytrain)
X_train_preds = model_rf.predict(Xtrain)
X_test_preds = model_rf.predict(Xtest)

# mean square/square root and R^2 values
print('train r2: {}'.format(r2_score(ytrain, X_train_preds)))
print('train mse: {}'.format(mean_squared_error(ytrain, X_train_preds)))
print('train rmse: {}'.format(sqrt(mean_squared_error(ytrain, X_train_preds))))

print()
print('test r2: {}'.format(r2_score(ytest, X_test_preds)))
print('test mse: {}'.format(mean_squared_error(ytest, X_test_preds)))
print('test rmse: {}'.format(sqrt(mean_squared_error(ytest, X_test_preds))))

### Random Forest Regression Model Performance Analysis ###

In [ ]:
# Scatter Observed Vs. Predicted using Random Forest Regressor

# scatter and best fit using target testing dataset and linear model calibrated using the features testing dataset
#ytest_plot = ytest.reset_index(drop=True)
fig,ax=plt.subplots(figsize=(20,10))
plt.grid()
plt.scatter(ytest, X_test_preds)
plt.title("Scatter Plot and Best-Fit of Observed vs. Predicted {}".format(climo_indice))
plt.xlabel('Observed {} ({})'.format(climo_indice, units))
plt.ylabel('Predicted {} ({})'.format(climo_indice, units))
lim = plt.axis()

# best fit using polyfit function
m, b = np.polyfit(ytest,X_test_preds, 1)
plt.plot(ytest, m*ytest+b, color='red')

In [ ]:
# Features Importance Analysis
# The feature importance determines which features as part of the training dataset are most important/valiable as predictors for the 
# target dataset

# feature_importances_ gives the importance of each feature in the order in which the features are arranged in training dataset.

# it is important to keep in mind that the importance of these features can become inflated artifically - indicating a false high
# predictive rating as a result of overfitting.

feature_names = list(X_data.columns) #rf[:-1].get_feature_names_out()

mdi_importances = pd.Series(
    model_rf.feature_importances_, index=feature_names
).sort_values(ascending=True)

#feature_names = model_rf[:-1].get_feature_names_out()

# plot to show feature importance
ax = mdi_importances.plot.barh()
ax.set_title("Random Forest Feature Importances")
ax.figure.tight_layout()

In [ ]:
# Permutation Importance Analysis
# this is defined as the decrease in a model score when a single feature value is randomly shuffled. 
# This procedure breaks the relationship between the feature and the target, thus the drop in the model 
# score is indicative of how much the model depends on the feature. This test is also not bias toward high-cardinality features

result = permutation_importance(
    model_rf,Xtest, ytest, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_data.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()